In [1]:
import re
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime

def getCommentCounts(newsurl):
    m = re.search('doc-i(.+).shtml',newsurl)
    newsid = m.group(1)
    comments = requests.get(commentURL.format(newsid))
    jd = json.loads(comments.text.strip('jsonp()'))
    return jd['result']['count']['total']

def getNewsDetail(newsurl):
    result = {}
    res = requests.get(newsurl)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    result['title'] = soup.select('.main-title')[0].text
    result['newssource'] = soup.select('.source')[0].text
    timesource = soup.select('.date')[0].text
    result['dt'] = datetime.strptime(timesource, '%Y年%m月%d日 %H:%M')
    result['article'] = ' '.join([p.text.strip() for p in soup.select('.article p')[:-1]])
    result['editor'] = soup.select('.show_author')[0].text.lstrip('责任编辑：')
    result['comments'] = getCommentCounts(newsurl)
    return result

def parseListlinks(url):
    newsdetails = []
    res = requests.get(url)
    jd = json.loads(res.text.replace('try{feedCardJsonpCallback(', '').replace(');}catch(e){};', '')) 
    for ent in jd['result']['data']: 
        newsdetails.append(getNewsDetail(ent['url']))
    return newsdetails

url = 'https://feed.sina.com.cn/api/roll/get?pageid=121&lid=1356&num=20&versionNumber=1.2.4&page={}&encode=utf-8&callback=feedCardJsonpCallback&_=1545358971124'

news_total = []
for i in range(1,2):
    newsurl = url.format(i)
    newsid = newsurl.split('/')[-1].rstrip('.shtml').lstrip('doc-i')
    commentURL = 'https://comment.sina.com.cn/page/info?version=1&format=json&channel=gn&newsid=comos-{}&group=undefined&compress=0&ie=utf-8&oe=utf-8&page=1&page_size=3&t_size=3&h_size=3&thread=1&callback=jsonp'
    commentURL.format(newsid)    
    newsary = parseListlinks(newsurl)
    news_total.extend(newsary)

news_total

[{'title': '新疆吐鲁番一能源化工企业发生闪爆事故 3死18伤',
  'newssource': '央视',
  'dt': datetime.datetime(2018, 12, 26, 18, 4),
  'article': '原标题：新疆吐鲁番一能源化工企业发生闪爆事故已致3死18伤 央视新闻客户端12月26日消息，12月25日16时50分许，新疆吐鲁番市托克逊县新疆新冶能源化工股份有限公司石灰回转窑在试车过程中发生闪爆事故，造成3人死亡、6人重伤、12人轻伤，当地迅速组织力量开展救援工作，全力救治受伤人员。 目前，伤员救治、事故调查及善后处置等工作正在进行中。',
  'editor': '余鹏飞 ',
  'comments': 0},
 {'title': '台媒突然“很兴奋” 只因大陆刚刚公布这件事',
  'newssource': '参考消息',
  'dt': datetime.datetime(2018, 12, 26, 17, 58),
  'article': '原标题：热点 | 台媒突然“很兴奋”！只因大陆刚刚公布这件事—— 国家发展改革委、商务部25日发布《市场准入负面清单（2018年版）》。媒体评价，这标志中国全面实施市场准入负面清单制度，负面清单以外的行业、领域、业务等，各类市场主体皆可依法平等进入。舆论注意到，这一消息引发诸多媒体关注，也包括台湾媒体。 就在有关“负面清单”的消息公布后，台湾《工商时报》、《旺报》等岛内媒体纷纷予以大篇幅报道。其中，台湾《工商时报》详尽梳理此次“负面清单”公布的主体、涉及行业、意义、与相关法律法规的比较、禁止准入类框架以及背景。同样，台湾《旺报》也详细列出了许可准入类及禁止准入类清单，并对大陆本轮的“进一步开放”予以肯定。 有分析认为，台湾媒体因大陆公布重大开放政策而“兴奋”，凸显岛内及台商对大陆市场的看重及期待。 对此，中国社会科学院台湾研究所研究员修春萍在接受参考消息网采访时表示，台湾媒体的反应在意料之中。当前，大陆已经成为全球最大的市场之一，对台商始终具有巨大的吸引力。在这一市场中占据一席之地，对台商而言十分重要。台商一直期盼在大陆能有更多的机会，如果大陆能提供鼓励或有利于台商发展的政策出台，对台商及台湾媒体而言自然会非常关注，而且台商也会努力抓住机遇。 舆论认为，多年来，尽管两

In [2]:
import pandas
df = pandas.DataFrame(news_total)
df.head(10)

,article,comments,dt,editor,newssource,title
0,原标题：新疆吐鲁番一能源化工企业发生闪爆事故已致3死18伤 央视新闻客户端12月26日消息，...,0,2018-12-26 18:04:00,余鹏飞,央视,新疆吐鲁番一能源化工企业发生闪爆事故 3死18伤
1,原标题：热点 | 台媒突然“很兴奋”！只因大陆刚刚公布这件事—— 国家发展改革委、商务部25...,0,2018-12-26 17:58:00,余鹏飞,参考消息,台媒突然“很兴奋” 只因大陆刚刚公布这件事
2,原标题：护航十年，听老兵讲述家国情怀：亚丁湾的海水是这样的滋味！ 2008年12月26日，中...,0,2018-12-26 17:51:00,余鹏飞,中央人民广播电台,护航十年听老兵讲家国情怀:亚丁湾的海水是这个味
3,原标题：国务院金融委七天内三度发声，释放哪些信号？ 划重点。 此前低调的国务院金融稳定发展委...,0,2018-12-26 17:45:00,余鹏飞,中国新闻网,国务院金融委七天内三度发声 释放哪些信号？
4,原标题：国家网信办：金融信息服务提供者不得散布虚假金融信息 第一条 为加强金融信息服务内容管...,0,2018-12-26 17:33:00,吴金明,澎湃新闻,国家网信办:金融信息服务提供者不得散布虚假信息
5,原标题：出门又遇见“座霸”、抢方向盘？不用怕，这回有人来管啦 12月23日，十三届全国人大常...,0,2018-12-26 17:23:00,余鹏飞,中央人民广播电台,出门又遇“座霸”抢方向盘不用怕 这回有人来管啦
6,原标题：未来，医院将按病种付费？北京、浙江金华已经开始实践 日前发布的《关于申报按疾病诊断相...,0,2018-12-26 17:18:00,吴金明,21世纪经济报道,医院将按病种付费？北京浙江金华已经开始实践
7,原标题：吉林检察机关依法分别对刘晓春、刚振涛提起公诉 新京报快讯 据最高人民检察院微信公众号...,0,2018-12-26 17:16:00,吴金明,新京报,吉林检察机关依法分别对两名厅官提起公诉
8,原标题：为啥别人家的孩子总有新玩具？现在，“小朋友圈”都在流行这个。。。 眼下，“共享”这个...,0,2018-12-26 17:11:00,余鹏飞,央视,为啥别人家的娃总有新玩具?小朋友圈也刮起共享风
9,原标题：中化集团原总经理蔡希有一审获刑12年 2018年12月26日上午，山东省泰安市中级人...,16,2018-12-26 17:07:00,余鹏飞,央视,中化集团原总经理蔡希有贪腐5367多万 一审判12年


In [5]:
df.to_excel('news.xlsx')